## Предварительные данные

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from sklearn.model_selection import ParameterGrid

C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
        
    return precision

def money_precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
        
    return precision

def recommendations(user, model, matrix, N=5):
    res = [id_to_itemid[rec] for rec in 
           model.recommend(userid=userid_to_id[user], 
                           user_items=matrix[userid_to_id[user]],
                           N=N, 
                           filter_already_liked_items=False, 
                           filter_items=[itemid_to_id[999999]], 
                           recalculate_user=True)[0]]
    return res   

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [5]:
merged_data = pd.merge(data, item_features, how='left')
merged_data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0,69,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,


In [6]:
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

### **Теоретические предположения**

**3 товара с акцией** 

Если нужно, чтобы в топ-5 наиболее подходящих товаров попало три товара с акцией, то или организуется акция на какие-то из этих пяти, или берем первые топ-N наиболее подходящих, в которые попадают первые три акционных, на данный момент товара. Соответственно, получаем 3 товара с акцией и два первых неакционных товара из топ-5. Акционные товары имеют дополнительно весовую характеристику и соответственно, если меняется процент, то меняется и характеристика, что сказывается на дальнейшем выборе подходящих товаров.

**1 новый товар**

Товар должен быть рекомендован из товаров с наибоее близкими показателями (например, косинусное сходство) к его потребительской корзине (возможно новый сорт или новый вид/подвид какого-либо продукта с новыми качествами). Также можно сделать этот товар с акцией, тем самым мотивировать на покупку (переплетение с предыдущим пунктом - 3 товара с акцией)

**1 товар для роста среднего чека**

Средний чек - это соотношение суммы покупок к количеству заказов (средний доход с чека). Можно добавить новый товар со скидкой (переплетение с предыдущим пунктом - 1 новый товар), но который всё равно будет добавлять стоимости его корзине. Как было сказано в первом пункте - минимум на 3%.

### **Практические действия**

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [9]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [11]:
params = {'factors': [40,50,60],
              'regularization': [0.05, 0.1, 0.15],
              'iterations': [3, 5, 7]}

grid = ParameterGrid(params)
metrics = {}
for i in range(len(grid)):
    model = AlternatingLeastSquares(factors=grid[i]['factors'], 
                                regularization=grid[i]['regularization'],
                                iterations=grid[i]['iterations'], 
                                calculate_training_loss=True, 
                                num_threads=6,
                                random_state=26)

    model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=False)
    params = f"w=none,f={grid[i]['factors']},r={grid[i]['regularization']},i={grid[i]['iterations']}"
    result[params] = result['user_id'].apply(lambda x: recommendations(x,model,sparse_user_item))
    metrics[params] = result.apply(lambda x: precision_at_k(x[params], x['actual']),1).mean()
    print(f'{i+1}/{len(grid)} completed;', end=' ')
    
 
    
    

1/27 completed; 2/27 completed; 3/27 completed; 4/27 completed; 5/27 completed; 6/27 completed; 7/27 completed; 8/27 completed; 9/27 completed; 10/27 completed; 11/27 completed; 12/27 completed; 13/27 completed; 14/27 completed; 15/27 completed; 16/27 completed; 17/27 completed; 18/27 completed; 19/27 completed; 20/27 completed; 21/27 completed; 22/27 completed; 23/27 completed; 24/27 completed; 25/27 completed; 26/27 completed; 27/27 completed; 

In [12]:
results = pd.DataFrame.from_dict(metrics, orient='index').T
results

,"w=none,f=40,r=0.05,i=3","w=none,f=40,r=0.1,i=3","w=none,f=40,r=0.15,i=3","w=none,f=40,r=0.05,i=5","w=none,f=40,r=0.1,i=5","w=none,f=40,r=0.15,i=5","w=none,f=40,r=0.05,i=7","w=none,f=40,r=0.1,i=7","w=none,f=40,r=0.15,i=7","w=none,f=50,r=0.05,i=3",...,"w=none,f=50,r=0.15,i=7","w=none,f=60,r=0.05,i=3","w=none,f=60,r=0.1,i=3","w=none,f=60,r=0.15,i=3","w=none,f=60,r=0.05,i=5","w=none,f=60,r=0.1,i=5","w=none,f=60,r=0.15,i=5","w=none,f=60,r=0.05,i=7","w=none,f=60,r=0.1,i=7","w=none,f=60,r=0.15,i=7"
0,0.199608,0.197649,0.197453,0.195397,0.193536,0.192948,0.193928,0.192262,0.192556,0.20284,...,0.193634,0.204505,0.202938,0.201861,0.198923,0.197747,0.197356,0.194515,0.193634,0.194515


In [13]:
print(results.idxmax(1)[0], results[results.idxmax(1)[0]])

w=none,f=60,r=0.05,i=3 0    0.204505
Name: w=none,f=60,r=0.05,i=3, dtype: float64


**TF-IDF weighed**

In [14]:
tfidf_user_item_matrix = tfidf_weight(user_item_matrix.T).T.tocsr()   # Применяется к item-user матрице ! 


In [15]:
params = {'factors': [50,100,150],
              'regularization': [0.001,0.01,0.1],
              'iterations': [10, 20, 50]}

grid = ParameterGrid(params)
metrics = {}
for i in range(len(grid)):
    model = AlternatingLeastSquares(factors=grid[i]['factors'], 
                                regularization=grid[i]['regularization'],
                                iterations=grid[i]['iterations'], 
                                calculate_training_loss=True, 
                                num_threads=6,
                                random_state=26)

    model.fit(tfidf_user_item_matrix,  # На вход item-user matrix
          show_progress=False)
    params = f"w=tfidf,f={grid[i]['factors']},r={grid[i]['regularization']},i={grid[i]['iterations']}"
    result[params] = result['user_id'].apply(lambda x: recommendations(x,model,sparse_user_item))
    metrics[params] = result.apply(lambda x: precision_at_k(x[params], x['actual']),1).mean()
    print(f'{i+1}/{len(grid)} ({metrics[params]});', end=' ')
    

1/27 (0.19275220372184135); 2/27 (0.19471106758080314); 3/27 (0.19666993143976494); 4/27 (0.19089128305582762); 5/27 (0.19118511263467192); 6/27 (0.19255631733594517); 7/27 (0.19275220372184135); 8/27 (0.19265426052889323); 9/27 (0.19382957884427035); 10/27 (0.18619000979431932); 11/27 (0.18775710088148875); 12/27 (0.19187071498530853); 13/27 (0.18667972575905978); 14/27 (0.18756121449559254); 15/27 (0.19079333986287955); 16/27 (0.1865817825661117); 17/27 (0.18785504407443684); 18/27 (0.1860920666013712); 19/27 (0.17952987267384918); 20/27 (0.18325171400587661); 21/27 (0.18667972575905978); 22/27 (0.178256611165524); 23/27 (0.18256611165524); 24/27 (0.1835455435847209); 25/27 (0.18011753183153772); 26/27 (0.18246816846229189); 27/27 (0.18354554358472086); 

In [16]:
tfidf_results = pd.DataFrame.from_dict(metrics, orient='index').T
print(tfidf_results.idxmax(1)[0], tfidf_results[tfidf_results.idxmax(1)[0]])

w=tfidf,f=50,r=0.1,i=10 0    0.19667
Name: w=tfidf,f=50,r=0.1,i=10, dtype: float64


**BM25 weighed**

In [17]:
bm25_user_item_matrix = bm25_weight(user_item_matrix.T).T.tocsr()

In [18]:
params = {'factors': [50,100,150],
              'regularization': [0.001,0.01,0.1],
              'iterations': [10, 20, 50]}

grid = ParameterGrid(params)
metrics = {}
for i in range(len(grid)):
    model = AlternatingLeastSquares(factors=grid[i]['factors'], 
                                regularization=grid[i]['regularization'],
                                iterations=grid[i]['iterations'], 
                                calculate_training_loss=True, 
                                num_threads=6,
                                random_state=26)

    model.fit(bm25_user_item_matrix,  # На вход item-user matrix
          show_progress=False)
    params = f"w=bm25,f={grid[i]['factors']},r={grid[i]['regularization']},i={grid[i]['iterations']}"
    result[params] = result['user_id'].apply(lambda x: recommendations(x,model,sparse_user_item))
    metrics[params] = result.apply(lambda x: precision_at_k(x[params], x['actual']),1).mean()
    print(f'{i+1}/{len(grid)} {metrics[params]};', end=' ')

1/27 0.19608227228207642; 2/27 0.197845249755142; 3/27 0.19960822722820765; 4/27 0.19696376101860924; 5/27 0.19980411361410383; 6/27 0.1992164544564153; 7/27 0.20039177277179238; 8/27 0.2007835455435847; 9/27 0.20195886385896178; 10/27 0.22624877571008817; 11/27 0.22859941234084233; 12/27 0.2300685602350637; 13/27 0.2240940254652302; 14/27 0.227130264446621; 15/27 0.22732615083251717; 16/27 0.22056807051909894; 17/27 0.22174338883447603; 18/27 0.22468168462291874; 19/27 0.23320274240940256; 20/27 0.23516160626836438; 21/27 0.2367286973555338; 22/27 0.2330068560235064; 23/27 0.23643486777668957; 24/27 0.23672869735553384; 25/27 0.22957884427032324; 26/27 0.23134182174338885; 27/27 0.2310479921645446; 

In [19]:
bm25_results = pd.DataFrame.from_dict(metrics, orient='index').T
print(bm25_results.idxmax(1)[0], bm25_results[bm25_results.idxmax(1)[0]])

w=bm25,f=150,r=0.1,i=20 0    0.236729
Name: w=bm25,f=150,r=0.1,i=20, dtype: float64


Все эти функции отправим затем в *utils.py*

**ДЗ**

1) Попытаться ответить на вопросы/выдвинуть гипотезы

*> e-mail мало кто читает, разве что рабочие. Зато все пользуются мессенджерами. +номер телефона при регистрации оставляют чаще, чем e-mail*<br>
*> слишком много уведомлений (письмо + смс + пуш + на чеке) тоже плохо, это отталкивает*<br>
*> реклама на чеке => увеличение расхода чековой бумаги => неэкологично*<br>
*> в смс не поместишь яркую картинку, что привлекает внимание; на чек тоже, чек никто не читает*<br>
*> пуш уведомления - только в приложении; приложениями пользуются только регулярные покупатели; чаще пользуются сайтом по мере необходимости*<br><br>

*> сортировать не только по вероятности; возможно какой-то товар люди не покупали, потому что дорогой, а со скидкой смогут себе позволить*<br>
*> рассылку отправлять 1 раз в неделю (чтобы ненапряжно); вечером, когда люди уже не заняты работой; перед выходными, когда у людей обычно есть время на закупки, в четверг или пятницу*<br>
*> возможно добавить проверку прошлых рекомендаций и исключать те, что были в прошлый раз*<br>
*> чтобы товары были разными, можно добавлять товары из разных категорий ИЛИ делать скидку только на ограниченное число товаров в рамках одной категории*<br><br>

**3 товара с акцией (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)**<br>
*последний вопрос не поняла*<br>
*с акцией лучше предлагать продукты изначально дорогие, т.к. изначально люди не могли их себе позволить, а со скидкой могут соблазниться*<br><br>

**1 новый товар (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)**<br>
*в дополнение к предложенному решению можно предлагать в принципе новый для магазина товар, как новинку; заодно и популяризуем/добавим инфы к новым товарам*<br><br>

**1 товар для роста среднего чека (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)**<br>
*можно предлагать товар, который юзер покупает чаще всего, но немножко дороже его обычного выбора*<br><br>




2) Доделать прошлые домашния задания


3) Прочитать статьи BM25/MatrixFactorization



4) Поэкспериментировать с ALS (grid-search)

**Links**

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/